<a href="https://colab.research.google.com/github/majeed76/Renewable-System-Optimization/blob/main/Average_Model/notebooks/Average_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import pandas as pd
import matplotlib.pyplot as plt

# === 1. Load solar irradiance data ===
solar_url = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/Timeseries_2022%20(1).csv"
df_solar = pd.read_csv(solar_url, skiprows=10, sep=",", engine='python', on_bad_lines='skip')

# Convert 'time' to datetime
df_solar['time'] = pd.to_datetime(df_solar['time'], format='%Y%m%d:%H%M', errors='coerce')
df_solar = df_solar.dropna(subset=['time'])

# Convert irradiance column to numeric
df_solar['G(i)'] = pd.to_numeric(df_solar['G(i)'], errors='coerce')

# Calculate daily and hourly averages
df_solar['date'] = df_solar['time'].dt.date
average_daily_irradiance = df_solar.groupby('date')['G(i)'].mean().mean()
average_hourly_irradiance = df_solar['G(i)'].mean()

<ipython-input-55-2781729045>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-55-2781729045>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [56]:
# === 2. Load electricity demand data ===
demand_url = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/consommation-annuelle-residentielle-par-adresse-2022.csv"
df_demand = pd.read_csv(demand_url, sep=";")

# Convert annual consumption to numeric (in MWh)
df_demand["Consommation annuelle totale de l'adresse (MWh)"] = pd.to_numeric(
    df_demand["Consommation annuelle totale de l'adresse (MWh)"], errors='coerce'
)
df_demand = df_demand.dropna(subset=["Consommation annuelle totale de l'adresse (MWh)"])

# Total and average demand
total_annual_kWh = df_demand["Consommation annuelle totale de l'adresse (MWh)"].sum() * 1000
average_daily_kWh = total_annual_kWh / 365
average_hourly_kWh = total_annual_kWh / 8760

In [57]:
# === 3. Display results ===
print("☀️ Average hourly solar irradiance:", round(average_hourly_irradiance, 2), "W/m²/hour")
print("📅 Average daily solar irradiance:", round(average_daily_irradiance, 2), "W/m²/day")
print("🔌 Total annual electricity demand:", round(total_annual_kWh, 2), "kWh/year")
print("📅 Average daily electricity demand:", round(average_daily_kWh, 2), "kWh/day")
print("⏰ Average hourly electricity demand:", round(average_hourly_kWh, 2), "kWh/hour")

☀️ Average hourly solar irradiance: 169.31 W/m²/hour
📅 Average daily solar irradiance: 169.31 W/m²/day
🔌 Total annual electricity demand: 1827071546.0 kWh/year
📅 Average daily electricity demand: 5005675.47 kWh/day
⏰ Average hourly electricity demand: 208569.81 kWh/hour


In [58]:
# Example parameters
panel_area = 2.56  # m²
panel_efficiency = 0.156
num_panels = 1000  # example number of panels

# Calculate hourly energy production per row
df_solar['E_solar_kWh'] = num_panels * panel_area * panel_efficiency * df_solar['G(i)'] / 1000

# Battery storage calculation
efficiency = 0.9
battery_capacity = 5_000_000  # in kWh
battery_level = 0
surplus = df_solar['E_solar_kWh'] - df_solar['demand_kWh']
battery_storage = []

for s in surplus:
    battery_level += s * efficiency if s > 0 else s  # Charge or discharge
    battery_level = max(0, min(battery_level, battery_capacity))  # Keep within limits
    battery_storage.append(battery_level)

df_solar['battery_storage'] = battery_storage

# Display results
print("🔋 Final battery level:", round(battery_level, 2), "kWh")
print("📊 Average surplus:", round(surplus.mean(), 2), "kWh/hour")


KeyError: 'demand_kWh'

In [ ]:
import pandas as pd
import numpy as np

# ── 1. Load & preprocess ──────────────────────────────────────────────────────
solar_url  = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/Timeseries_2022%20(1).csv"
demand_url = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/consommation-annuelle-residentielle-par-adresse-2022.csv"

df_solar = (
    pd.read_csv(solar_url, skiprows=10, sep=",", engine='python', on_bad_lines='skip')
      .assign(time=lambda d: pd.to_datetime(d['time'], format='%Y%m%d:%H%M', errors='coerce'))
      .dropna(subset=['time'])
)
df_solar['G(i)']      = pd.to_numeric(df_solar['G(i)'], errors='coerce').fillna(0)

df_demand = pd.read_csv(demand_url, sep=";")
df_demand["MWh"] = pd.to_numeric(df_demand["Consommation annuelle totale de l'adresse (MWh)"],
                                  errors='coerce').fillna(0)
total_kWh        = df_demand["MWh"].sum() * 1000
avg_hourly_kWh   = total_kWh / 8760
df_solar['demand_kWh'] = avg_hourly_kWh

# ── 2. Fixed parameters ───────────────────────────────────────────────────────
panel_area, panel_eff, batt_eff = 2.56, 0.156, 0.9

# ── 3. Function: count unmet hours for a given config ─────────────────────────
def unmet_hours(panels, capacity):
    prod    = panels * panel_area * panel_eff * df_solar['G(i)'] / 1000
    surplus = prod - df_solar['demand_kWh']
    batt, unmet = 0.0, 0
    for s in surplus:
        if s >= 0:
            batt += s * batt_eff
        else:
            if batt + s >= 0:
                batt += s
            else:
                batt, unmet = 0.0, unmet + 1
        batt = min(batt, capacity)
    return unmet

# ── 4. Search grid ─────────────────────────────────────────────────────────────
panel_vals   = np.arange(1_000, 51_0000, 1_000)      # 1k → 50k panels
storage_vals = np.arange(0, 5_000_001, 500_000)     # 0 → 5 MWh by 0.5 MWh

rows = []
for p in panel_vals:
    for c in storage_vals:
        u = unmet_hours(p, c)
        rows.append({'panels': p, 'storage_kWh': c, 'unmet_hours': u})

df = pd.DataFrame(rows)

# ── 5. Pick best combination ──────────────────────────────────────────────────
# 5.1 minimal unmet hours:
min_unmet = df['unmet_hours'].min()
cands     = df[df['unmet_hours'] == min_unmet]

# 5.2 among those, pick minimal storage:
min_storage = cands['storage_kWh'].min()
cands2      = cands[cands['storage_kWh'] == min_storage]

# 5.3 if still multiple, pick minimal panels:
best        = cands2.sort_values('panels').iloc[0]

print("Best configuration:")
print(f" • Panels       = {best.panels}")
print(f" • Storage (kWh)= {best.storage_kWh}")
print(f" • Unmet hours  = {best.unmet_hours}")


In [67]:
!pip install scikit-optimize
from google.colab import output
output.enable_custom_widget_manager()
from google.colab import output
output.disable_custom_widget_manager()

In [72]:
# 0) في أعلى النوتبوك تمكين دعم الـ FigureWidget
from google.colab import output
output.enable_custom_widget_manager()

# 1) استيراد المكتبات
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from skopt import gp_minimize
from skopt.space import Integer
from skopt.utils import use_named_args
from IPython.display import display

# 2) تحميل ومعالجة البيانات (كما عندك)
solar_url  = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/Timeseries_2022%20(1).csv"
demand_url = "https://raw.githubusercontent.com/majeed76/Renewable-System-Optimization/main/Average_Model/data/consommation-annuelle-residentielle-par-adresse-2022.csv"

df_solar = (
    pd.read_csv(solar_url, skiprows=10, sep=",", engine='python', on_bad_lines='skip')
      .assign(time=lambda d: pd.to_datetime(d['time'], format='%Y%m%d:%H%M', errors='coerce'))
      .dropna(subset=['time'])
)
df_solar['G(i)'] = pd.to_numeric(df_solar['G(i)'], errors='coerce').fillna(0)

df_demand = pd.read_csv(demand_url, sep=";")
df_demand["MWh"] = pd.to_numeric(
    df_demand["Consommation annuelle totale de l'adresse (MWh)"],
    errors='coerce'
).fillna(0)
total_kWh      = df_demand["MWh"].sum() * 1000
avg_hourly_kWh = total_kWh / 8760
df_solar['demand_kWh'] = avg_hourly_kWh

panel_area, panel_eff, batt_eff = 2.56, 0.156, 0.9

def unmet_hours(panels, capacity):
    prod    = panels * panel_area * panel_eff * df_solar['G(i)'] / 1000
    surplus = prod - df_solar['demand_kWh']
    batt, unmet = 0.0, 0
    for s in surplus:
        if s >= 0:
            batt += s * batt_eff
        else:
            if batt + s >= 0:
                batt += s
            else:
                batt, unmet = 0.0, unmet + 1
        batt = min(batt, capacity)
    return unmet

# 3) إنشاء الرسم التفاعلي مرة واحدة
fig = go.FigureWidget(
    data=[go.Scatter(
        x=[], y=[],
        mode='markers',
        marker=dict(size=[], color=[], colorscale='Viridis', showscale=True),
    )],
    layout=go.Layout(
        title="Bayesian Optimization Progress",
        xaxis_title='PV Panels',
        yaxis_title='Storage (kWh)',
    )
)
display(fig)

# 4) دالة الهدف
space = [
    Integer(1_000, 10_000_000,   name='panels'),
    Integer(0,     10_000_000, name='capacity_kWh'),
]

@use_named_args(space)
def objective(**p):
    u = unmet_hours(p['panels'], p['capacity_kWh'])
    penalty = 0.05 * p['capacity_kWh'] / 1000
    return u + penalty

# 5) callback لإضافة نقطة لكل تكرار
def plot_callback(res):
    p, c = res.x_iters[-1]
    val  = res.func_vals[-1]
    scatter = fig.data[0]
    scatter.x += (p,)
    scatter.y += (c,)
    scatter.marker.size += (8,)
    scatter.marker.color += (val,)

# 6) تشغيل GP minimizer مع callback
res = gp_minimize(
    func            = objective,
    dimensions      = space,
    acq_func        = "EI",
    n_calls         = 30,
    n_random_starts = 5,
    callback        = [plot_callback],
    random_state    = 0
)

best_p, best_c = res.x
best_u         = unmet_hours(best_p, best_c)
print(f"🔍 Best: panels={best_p}, storage={best_c} kWh, unmet_hours={best_u}")


FigureWidget({
    'data': [{'marker': {'color': [],
                         'colorscale': [[0.0, '#440154'], [0.1111111111111111,
                                        '#482878'], [0.2222222222222222,
                                        '#3e4989'], [0.3333333333333333,
                                        '#31688e'], [0.4444444444444444,
                                        '#26828e'], [0.5555555555555556,
                                        '#1f9e89'], [0.6666666666666666,
                                        '#35b779'], [0.7777777777777778,
                                        '#6ece58'], [0.8888888888888888,
                                        '#b5de2b'], [1.0, '#fde725']],
                         'showscale': True,
                         'size': []},
              'mode': 'markers',
              'type': 'scatter',
              'uid': '68f577e8-be1f-4d06-8ffc-49f9ac39a162',
              'x': [],
              'y': []}],
    'layout': {'template': '.

🔍 Best: panels=10000000, storage=7107832 kWh, unmet_hours=270


Support for third party widgets will remain active for the duration of the session. To disable support: